In [3]:
# The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
# P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
# Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

In [4]:
np.random.seed(40)

# Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
data = pd.read_csv("../data/airbnb-cleaned-mlflow.csv").iloc[3:]

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["price"], axis=1)
test_x = test.drop(["price"], axis=1)
train_y = train[["price"]]
test_y = test[["price"]]

alpha = 1.0
l1_ratio = 0.1

lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)


In [5]:
lr.fit(train_x, train_y)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.1,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=42, selection='cyclic', tol=0.0001, warm_start=False)

In [6]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

predicted_qualities = lr.predict(test_x)

(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)

Elasticnet model (alpha=1.000000, l1_ratio=0.100000):
  RMSE: 102.36429084520508
  MAE: 68.79511576580136
  R2: 0.4956017439244397


## Cross Validation via local GridSearchCV

In [7]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

parameters = {'alpha':(1.0, 0.5, 0.1), 'l1_ratio':[0.1, 0.5, 1.0]}
cv = GridSearchCV(lr, parameters, cv=5, return_train_score=True)
cv.fit(train_x, train_y)                        

GridSearchCV(cv=5, error_score='raise',
       estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.1,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=42, selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': (1.0, 0.5, 0.1), 'l1_ratio': [0.1, 0.5, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

## Log result of grid search to Tracking Server

In [8]:
cv.cv_results_

{'mean_fit_time': array([0.00445299, 0.00343242, 0.00388966, 0.00367379, 0.00364747,
        0.00423698, 0.00463667, 0.00440001, 0.00475092]),
 'mean_score_time': array([0.00066805, 0.0006494 , 0.00058093, 0.00053859, 0.00060663,
        0.00057282, 0.00054111, 0.00051818, 0.00053177]),
 'mean_test_score': array([0.47245022, 0.48718442, 0.51964441, 0.49067956, 0.502159  ,
        0.52088621, 0.51578271, 0.51869115, 0.54689143]),
 'mean_train_score': array([0.48432243, 0.49928995, 0.53221285, 0.50351894, 0.51520673,
        0.53462077, 0.52949625, 0.5327004 , 0.56317087]),
 'param_alpha': masked_array(data=[1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 0.1, 0.1, 0.1],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_l1_ratio': masked_array(data=[0.1, 0.5, 1.0, 0.1, 0.5, 1.0, 0.1, 0.5, 1.0],
              mask=[False, False, False, False, False, False, False, False,
                    Fals

In [10]:
import mlflow
import mlflow.sklearn
import os

def track(clf, name):
    params = list(clf.param_grid.keys())
    cv_results = clf.cv_results_
    rank = cv_results["rank_test_score"]

    for i in range(len(rank)):
        if i == clf.best_index_:
            run_name = "run %d (best run):" % i
        else:
            run_name = "run %d:" % i
        print(run_name)
        with mlflow.start_run(run_name=run_name, nested=True) as run:
            mlflow.log_param("folds", clf.cv)
            for param in params:
                print("  -", param, cv_results["param_%s" % param][i])
                mlflow.log_param(param, cv_results["param_%s" % param][i])
    
            mlflow.log_metric("rank_test_score" , cv_results["rank_test_score"][i])
            mlflow.log_metric("mean_train_score", cv_results["mean_train_score"][i])
            mlflow.log_metric("std_train_score",  cv_results["std_train_score"][i])
            mlflow.log_metric("mean_test_score",  cv_results["mean_test_score"][i])
            mlflow.log_metric("std_test_score",   cv_results["std_test_score"][i])
            if i == clf.best_index_:
                mlflow.sklearn.log_model(cv.best_estimator_, "model")
                local_path = os.path.join(".", "%s" % name)
                local_csv = os.path.join(local_path, "cv_results.csv")
                if not os.path.exists(local_path):
                    os.mkdir(local_path)
                pd.DataFrame(cv_results).sort_values(by='rank_test_score').to_csv(local_csv, index=False)
                mlflow.log_artifact(local_csv, "cv_results")
    print("Done")

**Local tracking server**

```bash
cd /opt/mlflow-tracking-server/
mkdir -p backend
mkdir -p artifacts
mlflow server --backend-store-uri ./backend --default-artifact-root ./artifacts/  --host 0.0.0.0
```

**In the project folder**

```bash
ln -s /opt/mlflow-tracking-server/artifacts artifacts
```

In [11]:
mlflow.set_tracking_uri("http://0.0.0.0:5000")

experiment="airbnb-jupyter"
mlflow.set_experiment(experiment)

track(cv, "airbnb")

run 0:
  - alpha 1.0
  - l1_ratio 0.1
run 1:
  - alpha 1.0
  - l1_ratio 0.5
run 2:
  - alpha 1.0
  - l1_ratio 1.0
run 3:
  - alpha 0.5
  - l1_ratio 0.1
run 4:
  - alpha 0.5
  - l1_ratio 0.5
run 5:
  - alpha 0.5
  - l1_ratio 1.0
run 6:
  - alpha 0.1
  - l1_ratio 0.1
run 7:
  - alpha 0.1
  - l1_ratio 0.5
run 8 (best run):
  - alpha 0.1
  - l1_ratio 1.0
Done


In [ ]:
#mlflow.set_tracking_uri("databricks://westeu")

#experiment="/Shared/experiments/airbnb-jupyter"
#mlflow.set_experiment(experiment)

#track(cv)